In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler#para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from ast import literal_eval
import time
import sys
import os
from scipy.signal import convolve2d
from functions.auxiliares import PaddingSameSize, MatrixFormat_To_Vector, RemoveOversizedMatrix, StandardSize_Padding, CNN_Features_Format, batch_format, KernelSize, StandardSize_InitialConvolution, StandardSize


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Esto garantiza que se ejecutara en GPU si esta disponible
print(device)

cpu


In [3]:
max_r = 30 ## Maximo numero de subconsultas
max_c = 40 ## Maximo numero de subcarateristicas

In [4]:
df_raw = pd.read_csv('D:/.Memoria/modelo/memoria_utfsm_deep_learning_models/Jupyter/dataset/test_example2.csv')
columns = list(df_raw.columns)
print(columns)
#  Eliminar los con tiempos de ejecución 0 o muy altos
df_clean = shuffle(df_raw[(df_raw['msec'] > 0) & (df_raw['msec'] < 1e10)])
df_clean.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean.csv',index=False)

### PASAR DE MSEC A SEGUNDOS
df_clean['msec'] = df_clean['msec'].apply(lambda x: x*0.001)

### HAY QUE CONVERTIR ALGUNAS COLUMNAS Q ESTAN EN PORCENTAJE A FLOAT
#df_clean['cpu_p'] = df_clean['cpu_p'].apply(lambda x: float(x.strip('%'))/100)
#df_clean['same_seg_p'] = df_clean['same_seg_p'].apply(lambda x: float(x.strip('%'))/100)
#df_clean['same_page_p'] = df_clean['same_page_p'].apply(lambda x: float(x.strip('%'))/100)
#df_clean['wait'] = df_clean['wait'].apply(lambda x: float(x.strip('%'))/100)
#df_clean['comp_read_p'] = df_clean['comp_read_p'].apply(lambda x: float(x.strip('%'))/100)
#df_clean['comp_clw'] = df_clean['comp_clw'].apply(lambda x: float(x.strip('%'))/100)

##SIN PASAR A FLOAT PERO QUITANDO EL %
df_clean['cpu_p'] = df_clean['cpu_p'].apply(lambda x: float(x.strip('%')))
df_clean['same_seg_p'] = df_clean['same_seg_p'].apply(lambda x: float(x.strip('%')))
df_clean['same_page_p'] = df_clean['same_page_p'].apply(lambda x: float(x.strip('%')))
df_clean['wait'] = df_clean['wait'].apply(lambda x: float(x.strip('%')))
df_clean['comp_read_p'] = df_clean['comp_read_p'].apply(lambda x: float(x.strip('%')))
df_clean['comp_clw'] = df_clean['comp_clw'].apply(lambda x: float(x.strip('%')))


# Convertir MatrixFormat de STR a np.array
df_clean['matrix_format'] = df_clean['matrix_format'].apply(lambda x: np.asarray(literal_eval(x)).astype(np.float32)) 


## shape de raw y clean
print(f'shape df_raw: {df_raw.shape}')
print(f'shape df_clean: {df_clean.shape}')


#new_df_clean,max_new_r,max_new_c,min_new_r,min_new_c,mean_new_r,mean_new_c = RemoveOversizedMatrix(df_clean,max_r,max_c)
new_df_clean = df_clean

#num_standard_rows = max_new_r
#num_standard_columns = max_new_c
num_standard_rows = 20
num_standard_columns = 32

#new_df_clean['matrix_format'] = df_clean['matrix_format'].apply(lambda x: StandardSize_Padding(x,num_standard_rows,num_standard_columns)) 
new_df_clean['matrix_format'] = df_clean['matrix_format'].apply(lambda x: StandardSize(x,num_standard_rows,num_standard_columns)) 
new_df_clean['matrix_format'] = new_df_clean['matrix_format'].apply(lambda x: x.astype(np.float32)) 


print(f'shape new_df_clean: {new_df_clean.shape}')

msk = np.random.rand(len(new_df_clean)) <= 0.8
df_train = new_df_clean[msk]
df_test = new_df_clean[~msk]

df_train.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_train.csv',index=False)
df_test.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_test.csv',index=False)



['unique_id', 'filename', 'sparql_file', 'profile', 'limit', 'msec', 'cpu_p', 'rnd', 'seq', 'same_seg_p', 'same_page_p', 'disk_reads', 'read_ahead', 'wait', 'comp_msec', 'comp_reads', 'comp_read_p', 'comp_messages', 'comp_clw', 'ql_id', 'ql_start_dt', 'ql_rt_msec', 'ql_rt_clocks', 'ql_client_ip', 'ql_user', 'ql_sqlstate', 'ql_error', 'ql_swap', 'ql_user_cpu', 'ql_sys_cpu', 'ql_params', 'ql_plan_hash', 'ql_c_clocks', 'ql_c_msec', 'ql_c_disk_reads', 'ql_c_disk_wait', 'ql_c_cl_wait', 'ql_cl_messages', 'ql_c_rnd_rows', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_same_parent', 'ql_thread_clocks', 'ql_disk_wait_clocks', 'ql_cl_wait_clocks', 'ql_pg_wait_clocks', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_messages', 'ql_message_bytes', 'ql_qp_threads', 'ql_memory', 'ql_memory_max', 'ql_lock_waits', 'ql_lock_wait_msec', 'ql_node_stat', 'ql_c_memory', 'ql_rows_affected', 'matrix_format', 'binary_tree']
shape df_raw: (29155, 62)
shape df_clean: (17099, 62)
shape new_df_clean:

In [5]:
features = [
    'matrix_format'
] 

## FEATURES - TRAIN Y TEST
X_df_train = df_train[features]
X_df_test = df_test[features]
X_numpy_train = X_df_train.to_numpy()
X_numpy_test = X_df_test.to_numpy()



## TARGETS - TRAIN Y TEST
y_df_train = df_train['msec']
y_df_test = df_test['msec']

y_numpy_train = y_df_train.to_numpy().astype(np.float32)
y_numpy_test = y_df_test.to_numpy().astype(np.float32)


print("-----------------------")
print(f'shape X_numpy_train: {X_numpy_train.shape}')
print(f'shape X_numpy_test: {X_numpy_test.shape}')
print(f'shape y_numpy_train: {y_numpy_train.shape}')
print(f'shape y_numpy_test: {y_numpy_test.shape}')
print("-----------------------")


# Pasarlos a Torch. 
#X_train = torch.from_numpy(X_numpy_train)
#X_test = torch.from_numpy(X_numpy_test)

#X_train = torch.tensor(df_train['matrix_format'].values)
#X_test = torch.tensor(df_test['matrix_format'].values)


X_train = CNN_Features_Format(X_numpy_train)
X_test = CNN_Features_Format(X_numpy_test)

y_train = torch.from_numpy(y_numpy_train)
y_test = torch.from_numpy(y_numpy_test)


# También pasar los targets de vector fila a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print(f'shape X_train: {X_train.shape}')
print(f'shape X_test: {X_test.shape}')
print(f'shape y_train: {y_train.shape}')
print(f'shape y_test: {y_test.shape}')
print("-----------------------")

-----------------------
shape X_numpy_train: (13767, 1)
shape X_numpy_test: (3332, 1)
shape y_numpy_train: (13767,)
shape y_numpy_test: (3332,)
-----------------------
shape X_train: torch.Size([13767, 1, 20, 32])
shape X_test: torch.Size([3332, 1, 20, 32])
shape y_train: torch.Size([13767, 1])
shape y_test: torch.Size([3332, 1])
-----------------------


In [6]:
y_train

tensor([[0.0020],
        [0.0040],
        [0.0010],
        ...,
        [0.0020],
        [0.0430],
        [0.0570]])

## Parametros

In [14]:
#n_samples_train, n_features_train = X_train.shape
#n_samples_test, n_features_test = X_test.shape
#
#hidden_size = 100000
#input_size = n_features_train 
#print(f'n_samples_train: {n_samples_train}')
#print(f'n_features_train: {n_features_train}')
#print(f'n_samples_test: {n_samples_test}')
#print(f'n_features_test: {n_features_test}')
#print(f'learning_rate: {learning_rate}')
#print(f'hidden_size: {hidden_size}')
#print(f'input_size: {input_size}')
learning_rate = 0.001
batches_size = 500
num_epochs = 5000
num_batches = math.ceil(X_train.shape[0]/batches_size)
batches = batch_format(X_train,y_train,batches_size)
for i in range(num_batches):
    print(batches["features"][i].shape)
    print(batches["labels"][i].shape)
print(X_train.shape)
print(y_train.shape)

torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])
torch.Size([500, 1, 20, 32])
torch.Size([500, 1])


## Modelo

In [15]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # SE DEFINEN LAS CAPAS A UTILIZAR, TANTO LAS CNN, FCL-ANN CY LAS DE POOLING
        self.conv1 = nn.Conv2d(1, 6, (3,5),stride=(1,1)) 
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(6, 12, (3,5),stride=(1,1)) 
        self.fc1 = nn.Linear(12*3*5, 162)
        self.fc2 = nn.Linear(162, 84)
        self.fc3 = nn.Linear(84, 42)
        self.fc4 = nn.Linear(42, 20)
        self.fc5 = nn.Linear(20, 1)
        
        #Se pueden agregar mas capas o mas neurones, cambiar tamaños etc.. pero siempre debo terminar con
        # una salida del tamaño que busco

    def forward(self, x):
        #print("input ",x.shape)
        x = F.leaky_relu(self.conv1(x), negative_slope=0.01, inplace=False)
        #print("conv1 ",x.shape)
        x = self.pool(x)
        #print("pooling ",x.shape)
        
        
        #print("input ",x.shape)
        x = F.leaky_relu(self.conv2(x), negative_slope=0.01, inplace=False)
        #print("conv2 ",x.shape)
        x = self.pool(x)
        #print("pooling2 ",x.shape)
        
        
        x = x.view(-1,12*3*5)           
        #print("view: ",x.shape)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.01, inplace=False)
        #print("fc1: ",x.shape)
        x = F.leaky_relu(self.fc2(x), negative_slope=0.01, inplace=False)
        #print("fc2: ",x.shape)
        x = F.leaky_relu(self.fc3(x), negative_slope=0.01, inplace=False)
        #print("fc3: ",x.shape)
        x = F.leaky_relu(self.fc4(x), negative_slope=0.01, inplace=False)
        #print("fc4: ",x.shape)
        x =self.fc5(x)
        #print("fc5: ",x.shape)
        #print("---------------------------------------------------------------")
        return x


model = ConvNet().to(device)

## Loss y Optimizer

In [16]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

## Training Loop

In [17]:
#print(num_batches)
for epoch in range(num_epochs):
    for bt in range(num_batches):
        y_hat = model(batches["features"][bt])
        loss = criterion(y_hat,batches["labels"][bt])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if bt+1 == num_batches and (epoch+1) % 100 == 0:
            print(f'epoch: [{epoch+1}/{num_epochs}], loss = {loss.item()}')
            
        

epoch: [100/5000], loss = 0.07055002450942993
epoch: [200/5000], loss = 0.10458239912986755
epoch: [300/5000], loss = 0.09148646146059036
epoch: [400/5000], loss = 0.32074078917503357
epoch: [500/5000], loss = 5.8807597160339355
epoch: [600/5000], loss = 0.1130882278084755
epoch: [700/5000], loss = 0.37568745017051697
epoch: [800/5000], loss = 0.27396291494369507
epoch: [900/5000], loss = 0.037042517215013504
epoch: [1000/5000], loss = 0.43837496638298035
epoch: [1100/5000], loss = 0.18599535524845123
epoch: [1200/5000], loss = 0.050574589520692825
epoch: [1300/5000], loss = 0.04174162819981575
epoch: [1400/5000], loss = 0.1337428241968155
epoch: [1500/5000], loss = 0.0305082555860281
epoch: [1600/5000], loss = 0.3281300663948059
epoch: [1700/5000], loss = 1.4097442626953125
epoch: [1800/5000], loss = 0.6369524598121643
epoch: [1900/5000], loss = 0.09455065429210663
epoch: [2000/5000], loss = 0.02452850714325905
epoch: [2100/5000], loss = 0.2893696129322052
epoch: [2200/5000], loss = 0

### Testing

In [18]:
# BASICO
print(X_test.shape)
print(y_test.shape)
print(batches_size)



torch.Size([3332, 1, 20, 32])
torch.Size([3332, 1])
500


In [19]:

batches_size_test = 1
num_batches_test = math.ceil(X_test.shape[0]/batches_size_test)
batches_test = batch_format(X_test,y_test,batches_size_test)
#for i in range(num_batches_test):
#    print(batches_test["features"][i].shape)
#    print(batches_test["labels"][i].shape)
#print(X_test.shape)
#print(y_test.shape)

y_pred = model(batches_test["features"][0])
y = batches_test["labels"][0]
print(y_pred)

print(y)

print()



tensor([[-0.0253]], grad_fn=<AddmmBackward>)
tensor([[0.0150]])



In [26]:
tol = 1 # segundos de tolerancia
num_aciertos = 0
num_test = float(y_test.shape[0])
with torch.no_grad():
    for bt in range(num_batches_test):
        #print(batches["features"][bt].shape)
        #print(batches["labels"][bt].shape)
        y_pred_tensor = model(batches_test["features"][bt])
        y_tensor = batches_test["labels"][bt]
        for i in range(y_tensor.shape[0]):
            y_pred = y_pred_tensor[i].item()
            y = y_tensor[i].item()
            #print(y, y_pred)
            if abs(y_pred-y) < tol:
                num_aciertos += 1 
            
    accuracy = num_aciertos/num_test

print(f'achuntes: [{num_aciertos}/{num_test}]')
print(f'accuracy: {accuracy*100}')
      
      

            

achuntes: [3237/3332.0]
accuracy: 97.14885954381752


In [21]:
print(f'achuntes: [{num_aciertos}/{num_test}]')
print(f'accuracy: {accuracy*100}')

achuntes: [3237/3332.0]
accuracy: 97.14885954381752
